# WB4.3 Introducción a SQL

En esta cuaderno de trabajo se explorarán la base de datos `music.db` mediante consultas de SQL. La base de datos contiene tres tablas:
* `songs`, que contiene un identificador `_id`, el título de la canción `title` y el album `album`. 
* `albums`, que contiene un identificador `_id`, el nombre del album `name` y el artista `artist`. 
* `artists`, que contiene un identificador `_id` y su nombre `name`. 

Completa las siguientes instrucciones de acuerdo a lo visto en el tema.

## 1. Acceso a los Datos con Queries.

1. Crea la conexión y un cursor a la base de datos.

In [1]:
import sqlite3
import pandas as pd

# 1. Crear la conexión a la base de datos
path = '/home/jose/Documentos/GitHub/VT-2.1-Connection-to-Structured-Database-and-APIs/music.db'
conn = sqlite3.connect(path)

# 2. Crear el cursor
cursor = conn.cursor()

print("Conexión exitosa y cursor listo.")

Conexión exitosa y cursor listo.


2. Explora todas las columnas en todas antes mencionadas tablas. Muestra los primeros 10 registros.

In [2]:
cursor.execute("SELECT * FROM songs LIMIT 10")

columnas = [descripcion[0] for descripcion in cursor.description]
print(f"\n--- Tabla: songs ---")
print(columnas) 
    
records = cursor.fetchall()
for row in records:
    print(row)


--- Tabla: songs ---
['_id', 'track', 'title', 'album']
(1, 2, "I Can't Quit You Baby", 343)
(2, 1, 'Taking the Easy Way Out Again', 311)
(3, 6, "Let's Have A Party", 260)
(4, 7, 'Flaming Telepaths', 104)
(5, 11, "Yearnin'", 37)
(6, 1, 'Bat Out Of Hell', 290)
(7, 5, 'Evil Woman', 143)
(8, 7, 'Rat Salad', 50)
(9, 8, 'Young is a World', 402)
(10, 3, 'Sam With The Showing Scalp Flat Top', 429)


In [3]:
cursor.execute("SELECT * FROM albums LIMIT 10")

columns = [descripcion[0] for descripcion in cursor.description]
print(f"\n--- Tabla: albums ---")
print(columns) 

records = cursor.fetchall()
for row in records:
    print(row)


--- Tabla: albums ---
['_id', 'name', 'artist']
(1, 'Tales of the Crown', 16)
(2, 'The Masquerade Ball', 16)
(3, 'Grace', 159)
(4, 'Behind Closed Doors', 147)
(5, 'Day & Age', 121)
(6, 'Sweet Fanny Adams', 172)
(7, 'Spem In Alium', 199)
(8, 'Night In The Ruts', 152)
(9, 'Impurity', 95)
(10, 'Concerto For Group and Orchestra', 196)


In [4]:
cursor.execute("SELECT * FROM artists LIMIT 10")


columns = [descripcion[0] for descripcion in cursor.description]
print(f"\n--- Tabla: artists ---")
print(columns)

records = cursor.fetchall()
for row in records:
    print(row)


--- Tabla: artists ---
['_id', 'name']
(1, 'Mahogany Rush')
(2, 'Elf')
(3, 'Mehitabel')
(4, 'Big Brother & The Holding Company')
(5, 'Roy Harper')
(6, 'Pat Benatar')
(7, 'Rory Gallagher')
(8, 'Iron Maiden')
(9, 'Blaster Bates')
(10, 'Procol Harum')


3. Iron Maiden tiene como `_id = 8`en la tabla de artistas. Emplea este id para identificar sus álbumes en la tabla `albums`. Observa que la columna `artist` hace referencia a los id de los artistas. 

In [5]:
cursor.execute("SELECT * FROM albums WHERE artist = 8")
columns = [descripcion[0] for descripcion in cursor.description]
records = cursor.fetchall()
for row in records:
    print(f"ID: {row[0]:<3}| Album Name: {row[1]:<25} | Artist ID: {row[2]}")

ID: 236| Album Name: The Number of the Beast   | Artist ID: 8
ID: 412| Album Name: Powerslave                | Artist ID: 8
ID: 420| Album Name: Seventh Son Of A Seventh Son | Artist ID: 8


4. Muestras las canciones del album "The Number of the Beast". Presenta tus resultados en orden alfabético.

In [6]:
cursor.execute("SELECT * FROM songs WHERE album = 236 ORDER BY title ASC")
columns = [descripcion[0] for descripcion in cursor.description]
records = cursor.fetchall()
for row in records:
    print(f"ID: {row[0]:<3}| Track: {row[1]:<25} | Title: {row[2]} | Album ID: {row[3]}")

ID: 3464| Track: 4                         | Title: 22 Acacia Avenue (1998 Digital Remaster) | Album ID: 236
ID: 2295| Track: 2                         | Title: Children Of The Damned (1998 Digital Remaster) | Album ID: 236
ID: 4515| Track: 7                         | Title: Gangland (1998 Digital Remaster) | Album ID: 236
ID: 3796| Track: 9                         | Title: Hallowed Be Thy Name (1998 Digital Remaster) | Album ID: 236
ID: 2332| Track: 1                         | Title: Invaders (1998 Digital Remaster) | Album ID: 236
ID: 195| Track: 6                         | Title: Run To The Hills (1998 Digital Remaster) | Album ID: 236
ID: 4333| Track: 5                         | Title: The Number Of The Beast (1998 Digital Remaster) | Album ID: 236
ID: 2690| Track: 3                         | Title: The Prisoner (1998 Digital Remaster) | Album ID: 236
ID: 275| Track: 8                         | Title: Total Eclipse (1998 Digital Remaster) | Album ID: 236


## 2. Agregaciones y Agrupamientos de Datos.

1. ¿Cuantas canciones, albumes y artistas se tienen registrados en la base de datos?

In [7]:
def execute_query(query):
    cursor.execute(query)
    columns = [descripcion[0] for descripcion in cursor.description]
    records = cursor.fetchall()
    return columns, records


total_content = """SELECT 
    (SELECT COUNT(*) FROM songs) AS total_songs,
    (SELECT COUNT(*) FROM albums) AS total_albums,
    (SELECT COUNT(*) FROM artists) AS total_artists;"""
execute_query(total_content)

(['total_songs', 'total_albums', 'total_artists'], [(5350, 439, 201)])

2. Agrupa la tabla albums por artista para identificar qué cuales son los 5 artistas que tienen más albumes.

In [8]:
artist_by_albums = """
SELECT 
    artists.name, 
    COUNT(albums._id) AS total_albums
FROM artists 
JOIN albums ON artists._id = albums.artist
GROUP BY artists._id, artists.name
ORDER BY total_albums DESC;
"""
df = pd.DataFrame(cursor.execute(artist_by_albums).fetchall(), columns=[descripcion[0] for descripcion in cursor.description])
df.head(5)



,name,total_albums
0,Black Sabbath,18
1,Axel Rudi Pell,15
2,Led Zeppelin,14
3,Aerosmith,13
4,Deep Purple,13


3. Presenta el nombre de los artistas identificados en el punto anterior.

In [9]:
artist_by_albums = """
SELECT 
    artists.name, 
    COUNT(albums._id) AS total_albums
FROM artists 
JOIN albums ON artists._id = albums.artist
GROUP BY artists._id, artists.name
ORDER BY total_albums DESC;
"""
df = pd.DataFrame(cursor.execute(artist_by_albums).fetchall(), columns=[descripcion[0] for descripcion in cursor.description])
df['name'].head(5)

0     Black Sabbath
1    Axel Rudi Pell
2      Led Zeppelin
3         Aerosmith
4       Deep Purple
Name: name, dtype: str

4. ¿Cuál es el album con más canciones? Investiga sobre este album en la red.

In [10]:
albums_by_songs = """
SELECT
    albums.name, 
    COUNT(songs._id) AS total_songs
FROM albums
JOIN songs ON albums._id = songs.album
GROUP BY albums._id, albums.name
ORDER BY total_songs DESC;
"""
df = pd.DataFrame(cursor.execute(albums_by_songs).fetchall(), columns=[descripcion[0] for descripcion in cursor.description])
df.head(5)

,name,total_songs
0,Cornology,72
1,Anyway The Wind Blows - The Anthology,50
2,Forty Licks,40
3,Super Lungs,37
4,Showbiz Kids,33


### Album Overview: Cornology

The album identified with the highest track count in your query is Cornology, a comprehensive box set by the British group The Bonzo Dog Doo-Dah Band. Released in 1992 by EMI Records, this anthology serves as a massive collection of the band's recorded output between 1967 and 1972. It contains a total of 72 tracks spread across three CDs, which explains why it appears as the top result in your database analysis.

### Key Features and Content

This collection is organized to cover the band's entire career, effectively aggregating five original studio albums into a single entry. The first disc covers their debut Gorilla, the second features Tadpoles and Keynsham, and the third includes their final album Let's Make Up and Be Friendly along with various rarities. Because it treats an entire discography as one product, it often functions as an outlier when searching for albums with the most songs.

### Artist Legacy and Impact

The Bonzo Dog Doo-Dah Band was a highly influential surrealist group that bridged the gap between 1960s British pop and avant-garde comedy. They were closely linked to the Monty Python troupe and even appeared in the Beatles' film Magical Mystery Tour. Their eclectic style, ranging from Jazz to Psychedelic Rock, left a lasting mark on music history, notably providing the name for the American indie-rock band Death Cab for Cutie.

## 3. Combinación de Tablas.

1. Muestra todas el nombre canciones del album "Seventh Son Of A Seventh Son" que tiene por '_id = 420'. Presenta la lista en el *track order* (`track`).

In [11]:
query = """
SELECT title
FROM songs
WHERE album = 420
ORDER BY track ASC;
"""

cursor.execute(query)
canciones = cursor.fetchall()

for i, (titulo,) in enumerate(canciones, start=1):
    print(f"{i:02d}. {titulo}")

01. Moonchild (1998 Digital Remaster)
02. Infinite Dreams (1998 Digital Remaster)
03. Can I Play With Madness (1998 Digital Remaster)
04. The Evil That Men Do (1998 Digital Remaster)
05. Seventh Son Of A Seventh Son (1998 Digital Remaster)
06. The Prophecy (1998 Digital Remaster)
07. The Clairvoyant (1998 Digital Remaster)
08. Only The Good Die Young (1998 Digital Remaster)


2. Obtén todas las canciones de Aeroesmith en orden alfabético. Solo incluye el título en la salida ¿Cuántas son?

In [12]:
query = """
SELECT s.title
FROM songs s
JOIN albums al ON s.album = al._id
JOIN artists ar ON al.artist = ar._id
WHERE ar.name LIKE '%Aerosmith%' OR ar.name LIKE '%Aeroesmith%'
ORDER BY s.title ASC;
"""

cursor.execute(query)
canciones = cursor.fetchall()

for (titulo,) in canciones:
    print(titulo)

print(f"\nTotal: {len(canciones)} canciones")

(Dulcimer Stomp) The Other Side
(Going Down) Love In An Elevator
(Hoodoo) Voodoo Medicine Man
(Water Song) Janie's Got A Gun
Adam's Apple
Adam's Apple
Ain't That A Bitch
Amazing
Amazing
Angel
Angel
Attitude Adjustment
Avant Garden
Back In The Saddle
Beyond Beautiful
Big Ten Inch Record
Blind Man
Bone To Bone (Coney Island White Fish Boy)
Boogie Man
Can't Stop Messin'
Cheese Cake
Chip Away The Stone
Chip Away The Stone
Chiquita
Come Together
Crash
Crazy
Crazy
Critical Mass
Cryin'
Cryin'
Darkness
Deuces Are Wild
Don't Get Mad Get Even
Dream On
Dream On
Drop Dead Gorgeous
Dude (Looks Like A Lady)
Dude (Looks Like A Lady)
Dude (Looks Like A Lady)- live
Eat The Rich
Eat The Rich
F.I.N.E.
Face
Fallen Angels
Falling In Love (Is Hard On The Knees)
Falling Off
Fever
Flesh
Fly Away From Here
Full Circle
Get A Grip
Girl Keeps Coming Apart
Gotta Love It
Gypsy Boots
Hangman Jury
Heart's Done Time
Hole In My Soul
I Ain't Got You
I'm Down
Intro
Jaded
Jailbait
Janie's Got A Gun
Just Push Play
Kiss You

## Referencias
* Actividad basada en SQLite-Exercises: https://github.com/IancuIulian/SQLite-Exercises
* SQLite SELECT Statement: https://www.sqlite.org/lang_select.html
* Cláusula ORDER BY: https://www.sqlite.org/lang_select.html#orderby
* Format Specification Mini-Language (f-strings): https://docs.python.org/3/library/string.html#format-specification-mini-language 
* List Comprehensions: https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions 